# IO Classes
This notebook shows basic use cases for the io classes. Most classes inherit from base FluentFile, and all make use of pythons context managment- this means that the "with PythonObject as instance:" syntax is required to read files. For more on context managment:

https://www.geeksforgeeks.org/context-manager-in-python/


### ReportFileOut
the report file out is intended to read files created by "report variables" in Fluent. These typically have a '.out' extension and the default naming is report-file-0.out. The ReportFileOut, like many files has a "readdf" method which will read the contents of the file into a pandas DataFrame. 

In [1]:
from wbfluentpy.io.classes import ReportFileOut
import os
data_folder = 'data_files'

with ReportFileOut(os.path.join(data_folder,'report-file-0.out')) as rfile:
    df = rfile.readdf()

print(df)

                 cs-htc     cs-temp     max-temp          p-in
Iteration                                                     
1             -0.514973  521.666480   522.494202  9.770612e+06
2             -0.072330  573.833122   574.603027  9.772086e+06
3             -0.032154  631.216434   632.006836  9.773973e+06
4             -0.057291  694.338079   695.151062  9.775655e+06
5             -0.071804  763.771889   764.609741  9.776419e+06
...                 ...         ...          ...           ...
817        13273.865069  837.622352  1202.924316  9.782954e+06
818        13273.882725  837.622031  1202.923828  9.782954e+06
819        13273.902107  837.621659  1202.923462  9.782954e+06
820        13273.918909  837.621313  1202.923096  9.782954e+06
821        13273.936812  837.620960  1202.922607  9.782954e+06

[821 rows x 4 columns]


### SolutionFile

The solution file class handles reading solution files created in Fluent. Solution files are transcripts, essentially recordings of the convergence activity of the numerical solution. These files have ".trn" extension with a default naming of "Solution.trn"

Solution files reading handles missing data by filling gaps with NaN. the size of the dataframe will be the total number of iterations, and any variable that begins later, or is reported at irregular intervals with be filled with NaN.

In [2]:
from wbfluentpy.io.classes import SolutionFile

#the first file here has no iterations, this will cause an AttributeError to be raised - left to the user to be handled
with SolutionFile(os.path.join(data_folder,'Solution.trn')) as sfile:
    df = sfile.readdf()
    

AttributeError: no columns found in solution file

In [ ]:
with SolutionFile(os.path.join(data_folder,'Solution-2.trn')) as sfile:
    df = sfile.readdf()
    

### PostDataFile
The PostDataFile reads data outputted from CFD post. The PostDataFile class can be treated as above by reading a single dataframe, or as a dictionary, reading multiple data frames if there are multiple surfaces in the post file. the PostDataFile works with multiple variable export, and assigns the keys of the dictionaries based on the names of the surfaces. These files usually have a .csv extension. 


In [ ]:
from wbfluentpy.io.classes import PostDataFile

with PostDataFile(os.path.join(data_folder,'yplus_and_htc_data.csv')) as pdfile:
    pdfile.readdf()
    
for key in pdfile.keys(): 
    print('Surface: {}'.format(key))
    print('Columns')
    print(pdfile[key].columns)
    print('')


### XYDataFile
The XYDataFile is designed to work with files exported from the "XY Plots" in Fluent. This is fairly similar to the format of PostDataFile with the representation of multiple data in a dictionary type manner. This does not handle missing data.


In [ ]:
from wbfluentpy.io.classes import XYDataFile

with XYDataFile(os.path.join(data_folder,'temp40HF.xy')) as xydfile:
    xydfile.readdf()
    
for key in xydfile.keys():
    print(key)
    print(xydfile[key])

### SurfacePointFile
the surface point file is meant for creating _and_ reading the results of surface points in Fluent. Points fall under the general category of "surfaces" in Fluent and are useful for extracting information from locations. The class only permits 3D data. The interface for creating input into fluent is through the classmethod "write_fluent_input_from_table" which takes arguments of 

X - the coordinates of the points, as a n x 3 size array
file_name: the file name to write the file to, as a string
export_variables: either a string or a list of strings of field variables we want to sample at the points 

The reading methods contain the standard readdf() method, which reads in the file. the method get_point_surface_data will read in the DataFrame, but will index the points based upon their position in the file



In [3]:
from wbfluentpy.io.classes import SurfacePointFile
import numpy as np

#creation of surface point file
X = np.linspace(0,1,10)[:,None]
X = np.concatenate([X,X,X],axis = 1)

#create input into Fluent
spf = SurfacePointFile.write_fluent_input_from_table(X,'surface_point_file','temperature')

In [ ]:
#reading output of a surface point file
from wbfluentpy.io.classes import SurfacePointFile

with SurfacePointFile(os.path.join(data_folder,'point_input.psf.out')) as spf:
    df = spf.readdf()
    print(df)
    df = spf.get_point_surface_data()
    print(df)

### SphereSliceFile
## WARNING: THIS CLASS IS UNDER ACTIVE DEVELOPMENT AND IS NOT RETURNING CORRECT RESULTS AS OF 11.20.2021

The SphereSliceFile is very similar to the SurfacePointFile, however it works with spheres rather than single points. Spheres consists of a list of datapoints rather than a single coordinates, and thus are slighlty more complicated to deal with. They also require specification of a radius in addition to their coordinates for input creation

In addition to the standard "readdf()" method, this class also contains the method get_sphere_surface_data(operator) where operator must be a callable that operates on the data of the sphere, for example: np.mean,np.max,np.min,np.median. This returns the result


In [5]:
from wbfluentpy.io.classes import SphereSliceFile

#create input for fluent
R = np.ones_like(X.shape[0])*1e-2
ssf = SphereSliceFile.write_fluent_input_from_table(X,R,'sphere_slice_file',['temperature','velocity'])

#read output from fluent

with SphereSliceFile(os.path.join(data_folder,'sphere.ssf.out')) as nssf:
    df = nssf.readdf()
    print(df)

Empty DataFrame
Columns: []
Index: [sphere-slice, sphere-0, 0.0, 0.0, 0.0, 0.01, sphere-slice, sphere-1, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.01, sphere-slice, sphere-2, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.01, sphere-slice, sphere-3, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.01, sphere-slice, sphere-4, 0.4444444444444444, 0.4444444444444444, 0.4444444444444444, 0.01, sphere-slice, sphere-5, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.01, sphere-slice, sphere-6, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.01, sphere-slice, sphere-7, 0.7777777777777777, 0.7777777777777777, 0.7777777777777777, 0.01, sphere-slice, sphere-8, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.01, sphere-slice, sphere-9, 1.0, 1.0, 1.0, 0.01, q, file/export/ascii, sphere_slice_file.out, sphere-0, sphere-1, sphere-2, sphere-3, sphere-4, sphere-5, sphere-6, sphere-7, sphere-8, sphere-9, yes, temp

b'Skipping line 75: expected 1 fields, saw 2\nSkipping line 79: expected 1 fields, saw 2\n'


In [2]:
#read output from fluent
from wbfluentpy.io.classes import SphereSliceFile
import numpy as np
import os
data_folder = 'data_files'

with SphereSliceFile(os.path.join(data_folder,'sphere.ssf.out')) as nssf:
    df = nssf.readdf()
    print(df.shape)
    df = nssf.get_sphere_surface_data(np.mean)
    print(df)
    

(824, 4)
    y-coordinate  z-coordinate  temperature
0      -0.000187     -0.000187   347.588209
1      -0.000176     -0.000176   468.205012
2      -0.000122     -0.000122   333.660848
3      -0.000121     -0.000121   441.438598
4      -0.000123     -0.000123   328.415587
5      -0.000123     -0.000123   413.743855
6      -0.000187     -0.000187   327.376457
7      -0.000188     -0.000188   390.075094
8      -0.000187     -0.000187   324.044867
9      -0.000189     -0.000189   383.076113
10     -0.000187     -0.000187   319.934659
11     -0.000188     -0.000188   374.363584
12     -0.000123     -0.000123   317.911928
13     -0.000125     -0.000125   364.452954
14     -0.000117     -0.000117   312.499126
15     -0.000117     -0.000117   356.311973
16     -0.000187     -0.000187   309.018169
17     -0.000182     -0.000182   348.888066
